In [ ]:
print('Start')

In [2]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
pd.options.display.max_columns = None
pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast
import warnings
import regex as re
warnings.filterwarnings('ignore')
from decimal import *
TWOPLACES = Decimal(10) ** -2
from natsort import natsorted
import ast
import time
today = time.strftime("%Y_%m_%d")
from enrich_dimensions.rounds import rounds, rounding_inch_feet,rounding_lbs,rounding_w,rounding_oz, rounding_lb,rounding_gal, re_extract, curate, round_string_float, clean_data,reg_ex,rounding_period_after_unit  
from enrich_dimensions.params import parameters, query_from_file
from enrich_dimensions.query_file import query_from_file 
from enrich_dimensions.custom import custom_field 

# parameters
customer_id = '5'
formatted_attribute = 'Origin'
customer_name='%bedbathandbeyond%'
buckets = """Office Chairs"""

strategy_version_input=767
attribute_id_input=2730

attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

df = query_from_file(file_name='./query/custom_fields.sql', params=params)
print(len(df))
print(today)

1208
2023_04_25


In [3]:
df.drop_duplicates(subset='external_id', inplace=True)
import collections
print([item for item, count in collections.Counter(df['external_id']).items() if count > 1])

[]


In [4]:
def three(usa,reg):
    trip=fr'''(?i){reg}|()''' 
    usa['m_name']=usa['product_name'].apply(lambda x: re_extract(trip,str(x)))
    usa['m_desc']=usa['long_desc'].apply(lambda x: re_extract(trip,str(x)))
    usa['m_custom']=usa['custom_fields'].apply(lambda x: re_extract(trip,str(x)))
    imported=usa[(usa['m_name'].astype(str)!='[]')|(usa['m_desc'].astype(str)!='[]')|(usa['m_custom'].astype(str)!='[]')]
    impor=usa[(usa['m_name'].astype(str)=='[]')&(usa['m_desc'].astype(str)=='[]')&(usa['m_custom'].astype(str)=='[]')]
    print('values: '+str(len(imported)))
    print('no values: '+str(len(impor)))
    return imported,impor

In [5]:
pat='''(imported(?!.material))|(made in Canada)|(made.?in.?(?:canada|turkey|china|france|india|germany|italy|the.?uk|poland|malaysia|belgium|vienna|pakistan))|(\b(?:united.?states|united.?states.?of.?america|USA(?!opoly)|AMerica|made.?in.?(?:us|u\.s\.))\b)'''
x,y=three(df,pat)

values: 122
no values: 1086


In [6]:
na=y[['external_id','value','product_name','long_desc','m_name','m_desc','m_custom']]#[0:500]
na['Q:origin']='n/a'
print(len(na))
match_na=na[['external_id','Q:origin']]

1086


In [7]:
pat='''(imported(?!.material))|(made in Canada)|(made.?in.?(?:canada|turkey|china|france|india|germany|italy|the.?uk|poland|malaysia|belgium|vienna|pakistan))'''
impor,non=three(x,pat)
pat='''(\b(?:united.?states|united.?states.?of.?america|USA(?!opoly)|AMerica|made.?in.?(?:us|u\.s\.))\b)'''
usa,non=three(x,pat)

values: 122
no values: 0
values: 0
no values: 122


In [8]:
impor['Q:origin']='Imported'
print(len(impor))
match_import=impor[['external_id','Q:origin']]

122


In [9]:
usa['Q:origin']='Made in the USA'
print(len(usa))
match_usa=usa[['external_id','Q:origin']]
match_usa

0


Empty DataFrame
Columns: [external_id, Q:origin]
Index: []

In [10]:
pat='''(imported(?!.material))|(made in Canada)|(made.?in.?(?:canada|turkey|china|france|india|germany|italy|the.?uk|poland|malaysia|belgium|vienna|pakistan))|(\b(?:united.?states|united.?states.?of.?america|USA(?!opoly)|AMerica|made.?in.?(?:us|u\.s\.))\b)'''
a,b=three(non,pat)

values: 122
no values: 0


In [11]:
# df1["isValid"] = 
usa.external_id.isin(impor.external_id)
# new_df = df1.loc[df1.isValid == True]

Series([], Name: external_id, dtype: bool)

In [12]:
print(len(df))
len(match_usa)+len(match_import)+len(match_na)

1208


1208

In [13]:
stop

NameError: name 'stop' is not defined

# send to the folder for upload

In [14]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute,matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BBBY - {attribute}-{get_df_name(matches)}-{today}-match_na.csv',index=False) 
    
looks_good('Bed Bath & Beyond', attribute,match_na,today)   

In [15]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute,matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BBBY - {attribute}-{get_df_name(matches)}-{today}-match_import.csv',index=False) 
    
looks_good('Bed Bath & Beyond', attribute,match_import,today)  

In [16]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute,matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BBBY - {attribute}-{get_df_name(matches)}-{today}-match_usa.csv',index=False) 
    
looks_good('Bed Bath & Beyond', attribute,match_usa,today)   